In [165]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

In [166]:
cwd = os.path.join(os.getcwd(), 'Group Coursework Brief-20221106', 'Data_Files', 'Data_Files')
dirName_trainData = os.path.join(cwd, 'epl-full-training.csv')

In [167]:
df_epl_train = pd.read_csv(dirName_trainData)
cols = ["Div","Date","HomeTeam","AwayTeam","FTHG", "FTAG","FTR","HTHG","HTAG","HTR","Referee","HS","AS", "HST","AST","HF","AF","HC","AC","HY","AY","HR","AR"]

df_epl_train = df_epl_train.loc[:, cols]
df_epl_train = df_epl_train.reset_index(drop=True)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_epl_train.head())

C:\Users\filip\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (98) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,17/08/2002,Blackburn,Sunderland,0.0,0.0,D,0.0,0.0,D,D Elleray,15.0,7.0,5.0,3.0,14.0,11.0,9.0,1.0,1.0,2.0,0.0,0.0
1,E0,17/08/2002,Charlton,Chelsea,2.0,3.0,A,2.0,1.0,H,G Barber,5.0,21.0,5.0,12.0,10.0,12.0,3.0,6.0,0.0,3.0,1.0,0.0
2,E0,17/08/2002,Everton,Tottenham,2.0,2.0,D,1.0,0.0,H,N Barry,13.0,10.0,9.0,5.0,18.0,4.0,10.0,5.0,1.0,1.0,0.0,0.0
3,E0,17/08/2002,Fulham,Bolton,4.0,1.0,H,3.0,1.0,H,A Wiley,13.0,3.0,6.0,1.0,16.0,12.0,7.0,4.0,1.0,2.0,0.0,0.0
4,E0,17/08/2002,Leeds,Man City,3.0,0.0,H,2.0,0.0,H,G Poll,13.0,18.0,8.0,10.0,13.0,13.0,2.0,7.0,1.0,1.0,0.0,0.0


In [168]:
# Transform the date column from strings into datetime objects
df_epl_train["Date"] = pd.to_datetime(df_epl_train["Date"], dayfirst=True)

In [169]:
df_epl_train.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,2002-08-17,Blackburn,Sunderland,0.0,0.0,D,0.0,0.0,D,...,5.0,3.0,14.0,11.0,9.0,1.0,1.0,2.0,0.0,0.0
1,E0,2002-08-17,Charlton,Chelsea,2.0,3.0,A,2.0,1.0,H,...,5.0,12.0,10.0,12.0,3.0,6.0,0.0,3.0,1.0,0.0
2,E0,2002-08-17,Everton,Tottenham,2.0,2.0,D,1.0,0.0,H,...,9.0,5.0,18.0,4.0,10.0,5.0,1.0,1.0,0.0,0.0
3,E0,2002-08-17,Fulham,Bolton,4.0,1.0,H,3.0,1.0,H,...,6.0,1.0,16.0,12.0,7.0,4.0,1.0,2.0,0.0,0.0
4,E0,2002-08-17,Leeds,Man City,3.0,0.0,H,2.0,0.0,H,...,8.0,10.0,13.0,13.0,2.0,7.0,1.0,1.0,0.0,0.0


AIM: A function that takes as input the date, HomeTeam and AwayTeam. It will filter the df_epl_train dataframe for matches between HomeTeam and AwayTeam that took place before the input date. Then take an average of the 
columns like HR, AR, etc. This will provide us with the past stats for games played in past between the two teams. We can then use these past stats (between the two teams) as features to input into the classifier.

First, filter the dataframe to include only matches where date is less than date specified and also only include matches where HomeTeam=input(HomeTeam) and AwayTeam=input(AwayTeam):

In [170]:
# This function will take as input a date, HomeTeam and AwayTeam and output a filtered dataframe where the matches shown are played before input data and match is between HomeTeam and AwayTeam

# For Example:
# date = "24/06/2020"
# HomeTeam = "Newcastle"
# AwayTeam = "Aston Villa"

def get_season_start_date(date):
    if date.month <= 7:
        return datetime(date.year-3, 8, 1)
    return datetime(date.year-2, 8, 1)

def filter_dataframe_by_bothteams_history(df, date, HomeTeam, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && HomeTeam=input(HomeTeam) && AwayTeam=input(AwayTeam)
    df_filtered = df_epl_train.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.HomeTeam==HomeTeam) & (df_filtered.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_filtered

def filter_dataframe_by_hometeam_recent_season(df, date, HomeTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && Date>input(first day of season) && HomeTeam=input(HomeTeam)
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.Date>get_season_start_date(date)) & (df_filtered.HomeTeam==HomeTeam)]

    # Return filtered dataframe
    return df_filtered

def filter_dataframe_by_awayteam_recent_season(df, date, AwayTeam):
    # Convert the input string date into datetime
    date = pd.to_datetime(date, dayfirst=True)

    # Filter the dataframe to include only rows where Date<input(Date) && Date>input(first day of season) && HomeTeam=input(HomeTeam)
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered.Date<date) & (df_filtered.Date>get_season_start_date(date)) & (df_filtered.AwayTeam==AwayTeam)]

    # Return filtered dataframe
    return df_filtered

# An example to see what the function does:
print(filter_dataframe_by_hometeam_recent_season(df_epl_train, "24/06/2020", "Newcastle"))

     Div       Date   HomeTeam          AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
5709  E0 2017-08-13  Newcastle         Tottenham   0.0   2.0   A   0.0   0.0   
5724  E0 2017-08-26  Newcastle          West Ham   3.0   0.0   H   1.0   0.0   
5744  E0 2017-09-16  Newcastle             Stoke   2.0   1.0   H   1.0   0.0   
5769  E0 2017-10-01  Newcastle         Liverpool   1.0   1.0   D   1.0   1.0   
5784  E0 2017-10-21  Newcastle    Crystal Palace   1.0   0.0   H   0.0   0.0   
5801  E0 2017-11-04  Newcastle       Bournemouth   0.0   1.0   A   0.0   0.0   
5824  E0 2017-11-25  Newcastle           Watford   0.0   3.0   A   0.0   2.0   
5853  E0 2017-12-09  Newcastle         Leicester   2.0   3.0   A   1.0   1.0   
5865  E0 2017-12-13  Newcastle           Everton   0.0   1.0   A   0.0   1.0   
5898  E0 2017-12-27  Newcastle          Man City   0.0   1.0   A   0.0   1.0   
5905  E0 2017-12-30  Newcastle          Brighton   0.0   0.0   D   0.0   0.0   
5923  E0 2018-01-13  Newcastle          

We now find the average of each of the columns that we need from this filtered dataframe e.g. HST, AST:

In [171]:
# This function takes as input the filtered dataframe from previous cell, features to average and a dictionary,
# it then appends an average of each feature to the dictionary

def average_columns(avg_features, filtered_df):
    for feature in avg_features.keys():
        df_col_means = filtered_df[feature].mean()
        avg_features[feature].append(df_col_means)

We now run the two functions on each row of the original dataframe to fill the dictionary with averages for each row

In [ ]:
# Run the two functions for each row of our df_epl_train dataframe to fill dictionary with AVG for each match
# NOTE: Some matches won't have past stats since the two teams may not have played against each other in past or we might not have the data

# These are the features we want to get averages for both teams
# avg_features_both = {
#                     "FTHG": [],
#                     "FTAG": [],
#                     "HTHG": [],
#                     "HTAG": [],
#                     "HS"  : [],
#                     "AS"  : [],
#                     "HST" : [],
#                     "AST" : [],
#                     "HF"  : [],
#                     "AF"  : [],
#                     "HC"  : [],
#                     "AC"  : [],
#                     "HY"  : [],
#                     "AY"  : [],
#                     "HR"  : [],
#                     "AR"  : []
#                 }
avg_features_both = {
                    "HS"  : [],
                    "AS"  : [],
                    "HST" : [],
                    "AST" : [],
                    "HF"  : [],
                    "AF"  : [],
                    "HC"  : [],
                    "AC"  : [],
                    "HY"  : [],
                    "AY"  : [],
                    "HR"  : [],
                    "AR"  : []
                }

# These are the features we want to get averages for home team
# avg_features_home = {
#                     "FTHG": [],
#                     "HTHG": [],
#                     "HS"  : [],
#                     "HST" : [],
#                     "HF"  : [],
#                     "HC"  : [],
#                     "HY"  : [],
#                     "HR"  : [],
#                 }
avg_features_home = {
                    "HS"  : [],
                    "HST" : [],
                    "HF"  : [],
                    "HC"  : [],
                    "HY"  : [],
                    "HR"  : [],
                }

# These are the features we want to get averages for away team
# avg_features_away = {
#                     "FTAG": [],
#                     "HTAG": [],
#                     "AS"  : [],
#                     "AST" : [],
#                     "AF"  : [],
#                     "AC"  : [],
#                     "AY"  : [],
#                     "AR"  : []
#                   }
avg_features_away = {
                    "AS"  : [],
                    "AST" : [],
                    "AF"  : [],
                    "AC"  : [],
                    "AY"  : [],
                    "AR"  : []
                }


# Run the two functions on each row of the df_epl_train and fill the dictionary
# For each row in the dataframe
for index, row in df_epl_train.iterrows():
    # Filter the dataframe to only show matches played between those teams and before the certain date
    df_epl_train_average_bothteams_history = filter_dataframe_by_bothteams_history(df_epl_train, row["Date"],row["HomeTeam"],row["AwayTeam"])
    df_epl_train_average_hometeam_recent_season = filter_dataframe_by_hometeam_recent_season(df_epl_train, row["Date"],row["HomeTeam"])
    df_epl_train_average_awayteam_recent_season = filter_dataframe_by_awayteam_recent_season(df_epl_train, row["Date"],row["AwayTeam"])
    # Get averages from the filtered dataframe and add the the dictionary
    average_columns(avg_features_both, df_epl_train_average_bothteams_history)
    average_columns(avg_features_home, df_epl_train_average_hometeam_recent_season)
    average_columns(avg_features_away, df_epl_train_average_awayteam_recent_season)

In [ ]:
print(avg_features_home)

Add these average feature lists in the dictionary back into the original dataframe as columns:

In [ ]:
# Add a column for each of these feature averages using our list of values from the dictionary
df_epl_train_updated = df_epl_train.copy()

for feature in avg_features_both.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_both[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_HISTORY"] = feature_vals

for feature in avg_features_home.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_home[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals
    
for feature in avg_features_away.keys():
    # Get the list of averages for a certain feature from the dicitonary
    feature_vals = avg_features_away[feature]
    # Add the list of averages into the dataframe for that certain feature
    df_epl_train_updated[feature + "_AVG"] = feature_vals

This is the new dataframe with the added columns with past average statistics for each row:

In [ ]:
# Now this dataframe contains our original data + the average of the past stats for each row
print(df_epl_train_updated)

NOTE: Some rows have nan values, these are matches where teams might not have played each other before (or we dont have the past match data for them)

So we need to remove these rows (from the dataframe) where there are nan values. This is required to run the classifier training

In [ ]:
# NOTE: We must remove the rows in the dataframe where the average values of stats/features are 'nan';
# we get these values because either the two teams have not played a match in the past OR because we have
# not got the past stats for these matches. We cannot use the 'nan' values for the classifier training and 
# hence have to remove these rows. We can then train a classifier using this final dataframe.

# In the final model/classifier, in the case where we DO NOT have these past stats of the two teams playing, 
# we need to switch back to using the OLD classifier which only took the 4 basic fetaures: day, month, 
# HomeTeam and AwayTeam.

# In the case where we DO have these past stats for two teams playing each other, we can use this model/classifier 
# and input the features like HST_AVG and AST_AVG. We would find these by using the filter_dataframe() and 
# average_columns() functions to find them for any two specific teams playing each other on some date.

# Remove any rows with nan
df_epl_train_final = df_epl_train_updated.dropna()
df_epl_train_final

In [ ]:
# Turn the catergorical data into labels using same method from before
df_epl_train_final["AwayTeam_Enc"] = df_epl_train_final["AwayTeam"].astype("category").cat.codes
df_epl_train_final["HomeTeam_Enc"] = df_epl_train_final["HomeTeam"].astype("category").cat.codes

# Transform the date column into day and month columns and Add into dataframe (Extract days & months from date)
df_epl_train_final["Date"] = pd.to_datetime(df_epl_train_final["Date"])
df_epl_train_final["Day"] = df_epl_train_final["Date"].dt.day
df_epl_train_final["Month"] = df_epl_train_final["Date"].dt.month 
df_epl_train_final["Year"] = df_epl_train_final["Date"].dt.year

# Check the final updated dataframe
df_epl_train_final

Create our X and y matrix and split into a training and test set:

In [ ]:
# Create the input features matrix X (made of day, month, HomeTeam, AwayTeam, FTHG_AVG, FTAG_AVG, etc)
# Create the output values y vector (made of FTR)
# Take these values from the transformed dataframe

# X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc','FTHG_AVG','FTAG_AVG','HTHG_AVG','HTAG_AVG','HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
X = df_epl_train_final.loc[:,['Day', 'Month', 'HomeTeam_Enc', 'AwayTeam_Enc',
                              'HS_HISTORY','AS_HISTORY','HST_HISTORY','AST_HISTORY','HF_HISTORY','AF_HISTORY','HC_HISTORY','AC_HISTORY','HY_HISTORY','AY_HISTORY','HR_HISTORY','AR_HISTORY',
                              'HS_AVG','AS_AVG','HST_AVG','AST_AVG','HF_AVG','AF_AVG','HC_AVG','AC_AVG','HY_AVG','AY_AVG','HR_AVG','AR_AVG']].values
y = df_epl_train_final.loc[:,'FTR'].values

# Split the training data in a 80-20 split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=22)

# Encode the y output values as well
FTR_encoder = LabelEncoder()
y_train = FTR_encoder.fit_transform(y_train);

Now we can test using the different classifiers:

In [ ]:
# Create an empty Tree model
DT_Model = DecisionTreeClassifier(random_state=42)
# Fit the model using training data
DT_Model.fit(X_train, y_train)
# Make predictions using the model we have created
DT_predictions_test = DT_Model.predict(X_test)
# Reconverting prediction values (i.e. 0, 1 or 2) back into (H, D or A) using the FTR_encoder defined in earlier cell
DT_predictions_test = FTR_encoder.inverse_transform(DT_predictions_test)

In [ ]:
print(accuracy_score(DT_predictions_test, y_test))
print(classification_report(DT_predictions_test, y_test))

In [ ]:
# Create an empty KNN model
KNN_Model = KNeighborsClassifier(n_neighbors=6)
# Fit the model using training data
KNN_Model.fit(X_train, y_train)
# Make predictions using the model we have created
KNN_predictions_test = KNN_Model.predict(X_test)
KNN_predictions_test = FTR_encoder.inverse_transform(KNN_predictions_test)

In [ ]:
print(accuracy_score(KNN_predictions_test, y_test))
print(classification_report(KNN_predictions_test, y_test))

In [ ]:
# Create an empty Random Forest model
RF_Model = RandomForestClassifier(n_estimators=50, random_state=42)
# Fit the model using training data
RF_Model.fit(X_train, y_train)
# Make predictions using the model we have created
RF_predictions_test = RF_Model.predict(X_test)
RF_predictions_test = FTR_encoder.inverse_transform(RF_predictions_test)

In [ ]:
print(accuracy_score(RF_predictions_test, y_test))
print(classification_report(RF_predictions_test, y_test))